# Running Tune experiments with Skopt

In this tutorial we introduce Skopt, while running a simple Ray Tune experiment. Tune’s Search Algorithms integrate with Skopt and, as a result, allow you to seamlessly scale up a Skopt optimization process - without sacrificing performance.

Scikit-Optimize, or skopt, is a simple and efficient library to optimize expensive and noisy black-box functions, e.g. large-scale ML experiments. It implements several methods for sequential model-based optimization. Noteably, skopt does not perform gradient-based optimization, and instead uses computationally cheap surrogate models to
approximate the expensive function. In this example we minimize a simple objective to briefly demonstrate the usage of Skopt with Ray Tune via `SkOptSearch`. It's useful to keep in mind that despite the emphasis on machine learning experiments, Ray Tune optimizes any implicit or explicit objective. Here we assume `scikit-opitmize==0.8.1` library is installed. To learn more, please refer to the [Scikit-Optimize website](https://scikit-optimize.github.io).


In [1]:
# !pip install ray[tune]
!pip install scikit-optimize==0.8.1
!pip install sklearn==0.18.2

  Using cached scikit_optimize-0.8.1-py2.py3-none-any.whl (101 kB)
ERROR: Could not find a version that satisfies the requirement sklearn==0.18.2 (from versions: 0.0)
ERROR: No matching distribution found for sklearn==0.18.2


Click below to see all the imports we need for this example.
You can also launch directly into a Binder instance to run this notebook yourself.
Just click on the rocket symbol at the top of the navigation.

In [2]:
import time
from typing import Dict, Optional, Any

import ray

import skopt
from ray import tune
from ray.air import session
from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.skopt import SkOptSearch

In [3]:
ray.init(configure_logging=False)

Python version:,3.7.7
Ray version:,3.0.0.dev0
Dashboard:,http://127.0.0.1:8268


Let's start by defining a simple evaluation function. Again, an explicit math formula is queried here for demonstration, yet in practice this is typically a black-box function-- e.g. the performance results after training an ML model. We artificially sleep for a bit (`0.1` seconds) to simulate a long-running ML experiment. This setup assumes that we're running multiple `step`s of an experiment while tuning three hyperparameters, namely `width`, `height`, and `activation`.

In [4]:
def evaluate(step, width, height, activation):
    time.sleep(0.1)
    activation_boost = 10 if activation=="relu" else 0
    return (0.1 + width * step / 100) ** (-1) + height * 0.1 + activation_boost

Next, our `objective` function to be optimized takes a Tune `config`, evaluates the `score` of your experiment in a training loop,
and uses `session.report` to report the `score` back to Tune.

In [5]:
def objective(config):
    for step in range(config["steps"]):
        score = evaluate(step, config["width"], config["height"], config["activation"])
        session.report({"iterations": step, "mean_loss": score})

Next we define a search space. The critical assumption is that the optimal hyperparamters live within this space. Yet, if the space is very large, then those hyperparameters may be difficult to find in a short amount of time.

In [6]:
search_space = {
    "steps": 100,
    "width": tune.uniform(0, 20),
    "height": tune.uniform(-100, 100),
    "activation": tune.choice(["relu", "tanh"]),
}

The search algorithm is instantiated from the `SkOptSearch` class. We also constrain the the number of concurrent trials to `4` with a `ConcurrencyLimiter`.

In [7]:
algo = SkOptSearch()
algo = ConcurrencyLimiter(algo, max_concurrent=4)


The number of samples is the number of hyperparameter combinations that will be tried out. This Tune run is set to `1000` samples.
(you can decrease this if it takes too long on your machine).

In [8]:
num_samples = 1000

In [9]:
# We override here for our smoke tests.
num_samples = 10

Finally, we run the experiment to `"min"`imize the "mean_loss" of the `objective` by searching `search_config` via `algo`, `num_samples` times. This previous sentence is fully characterizes the search problem we aim to solve. With this in mind, notice how efficient it is to execute `tuner.fit()`.

In [10]:
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        num_samples=num_samples,
    ),
    param_space=search_space,
)
results = tuner.fit()

Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,activation,height,width,loss,iter,total time (s),iterations,neg_mean_loss
objective_0ec1ec00,TERMINATED,127.0.0.1:48435,tanh,10.9976,8.60092,1.21584,100,10.941,99,-1.21584
objective_107c17fa,TERMINATED,127.0.0.1:48446,relu,49.5455,11.8085,15.0394,100,11.5877,99,-15.0394
objective_107d75f0,TERMINATED,127.0.0.1:48447,tanh,-7.6131,3.10991,-0.446727,100,11.7109,99,0.446727
objective_107ef722,TERMINATED,127.0.0.1:48448,relu,-75.8366,5.30975,2.60302,100,11.6501,99,-2.60302
objective_17033b12,TERMINATED,127.0.0.1:48470,tanh,-89.1891,12.1463,-8.83643,100,10.7255,99,8.83643
objective_18fbb20a,TERMINATED,127.0.0.1:48475,tanh,33.5568,9.75437,3.45818,100,10.7563,99,-3.45818
objective_190ab4d0,TERMINATED,127.0.0.1:48478,relu,-65.148,4.19546,3.7203,100,10.7123,99,-3.7203
objective_191ca3de,TERMINATED,127.0.0.1:48481,tanh,-13.4892,6.74078,-1.20128,100,13.219,99,1.20128
objective_1eec012e,TERMINATED,127.0.0.1:48494,tanh,-50.2819,14.8287,-4.96053,100,13.2232,99,4.96053
objective_20e93604,TERMINATED,127.0.0.1:48502,tanh,-89.9051,8.24648,-8.86951,100,10.7925,99,8.86951


Result for objective_0ec1ec00:
  date: 2022-07-22_15-46-26
  done: false
  experiment_id: c3e1e5ac1d6c4d67878bf793d020f99e
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 11.099760178357887
  neg_mean_loss: -11.099760178357887
  node_ip: 127.0.0.1
  pid: 48435
  time_since_restore: 0.10384106636047363
  time_this_iter_s: 0.10384106636047363
  time_total_s: 0.10384106636047363
  timestamp: 1658501186
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 0ec1ec00
  warmup_time: 0.003099679946899414
  
Result for objective_107c17fa:
  date: 2022-07-22_15-46-29
  done: false
  experiment_id: f2a0e8ad4ea64c489424406b75b5c051
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 24.95455186350225
  neg_mean_loss: -24.95455186350225
  node_ip: 127.0.0.1
  pid: 48446
  time_since_restore: 0.10309791564941406
  time_this_iter_s: 0.10309791564941406
  time_total_s: 0.10309791564941406
  timestamp: 16

Result for objective_107d75f0:
  date: 2022-07-22_15-46-41
  done: true
  experiment_id: a44e7e5deb2e4bc68b31ca0466db50f5
  experiment_tag: 3_activation=tanh,height=-7.6131,steps=100,width=3.1099
  hostname: Kais-MacBook-Pro.local
  iterations: 99
  iterations_since_restore: 100
  mean_loss: -0.44672713582845847
  neg_mean_loss: 0.44672713582845847
  node_ip: 127.0.0.1
  pid: 48447
  time_since_restore: 11.710935115814209
  time_this_iter_s: 0.1116342544555664
  time_total_s: 11.710935115814209
  timestamp: 1658501201
  timesteps_since_restore: 0
  training_iteration: 100
  trial_id: 107d75f0
  warmup_time: 0.0029137134552001953
  
Result for objective_18fbb20a:
  date: 2022-07-22_15-46-43
  done: false
  experiment_id: 5e399bc125a548739dd433f4153019fe
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 13.355684066332913
  neg_mean_loss: -13.355684066332913
  node_ip: 127.0.0.1
  pid: 48475
  time_since_restore: 0.10402607917785645
  time_this

Result for objective_190ab4d0:
  date: 2022-07-22_15-46-54
  done: true
  experiment_id: 1b007a40cf6d425085379a4314739912
  experiment_tag: 7_activation=relu,height=-65.1480,steps=100,width=4.1955
  hostname: Kais-MacBook-Pro.local
  iterations: 99
  iterations_since_restore: 100
  mean_loss: 3.7203002216299694
  neg_mean_loss: -3.7203002216299694
  node_ip: 127.0.0.1
  pid: 48478
  time_since_restore: 10.712332725524902
  time_this_iter_s: 0.10630083084106445
  time_total_s: 10.712332725524902
  timestamp: 1658501214
  timesteps_since_restore: 0
  training_iteration: 100
  trial_id: 190ab4d0
  warmup_time: 0.0028760433197021484
  
Result for objective_20e93604:
  date: 2022-07-22_15-46-56
  done: false
  experiment_id: e2c12d5518de42acb5f8413171242ef8
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 1.0094856921511255
  neg_mean_loss: -1.0094856921511255
  node_ip: 127.0.0.1
  pid: 48502
  time_since_restore: 0.10361671447753906
  time_this

We now have hyperparameters found to minimize the mean loss.

In [11]:
print("Best hyperparameters found were: ", results.get_best_result().config)

Best hyperparameters found were:  {'steps': 100, 'width': 8.246480084328711, 'height': -89.90514307848873, 'activation': 'tanh'}


## Providing an initial set of hyperparameters

While defining the search algorithm, we may choose to provide an initial set of hyperparameters that we believe are especially promising or informative, and
pass this information as a helpful starting point for the `SkOptSearch` object. We also can pass the known rewards for these initial params to save on unnecessary computation.

In [12]:
initial_params = [
    {"width": 10, "height": 0, "activation": "relu"},
    {"width": 15, "height": -20, "activation": "tanh"}
]
known_rewards = [-189, -1144]

Now the `search_alg` built using `SkOptSearch` takes `points_to_evaluate`.

In [13]:
algo = SkOptSearch(points_to_evaluate=initial_params)
algo = ConcurrencyLimiter(algo, max_concurrent=4)

And again run the experiment, this time with initial hyperparameter evaluations:

In [15]:
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        num_samples=num_samples,
    ),
    param_space=search_space,
)
results = tuner.fit()

Trial name,status,loc,activation,height,width,loss,iter,total time (s),iterations,neg_mean_loss
objective_56a79fce,TERMINATED,127.0.0.1:48590,relu,0,10,10.1,100,10.9393,99,-10.1
objective_583ee2ca,TERMINATED,127.0.0.1:48605,tanh,-20,15,-1.93311,100,11.6366,99,1.93311
objective_58405952,TERMINATED,127.0.0.1:48606,relu,-87.8025,19.608,1.271,100,11.752,99,-1.271
objective_5841ebe6,TERMINATED,127.0.0.1:48607,relu,49.51,15.0894,15.0175,100,11.6761,99,-15.0175
objective_5ec4da6e,TERMINATED,127.0.0.1:48629,relu,76.0837,13.7995,17.681,100,10.7667,99,-17.681
objective_60d7fb56,TERMINATED,127.0.0.1:48634,relu,-50.4534,16.3133,5.0162,100,10.75,99,-5.0162
objective_60e6657e,TERMINATED,127.0.0.1:48637,tanh,-66.5228,11.4506,-6.56484,100,10.759,99,6.56484
objective_60f84e42,TERMINATED,127.0.0.1:48640,tanh,99.5965,9.04425,10.0701,100,10.71,99,-10.0701
objective_66bdb9e8,TERMINATED,127.0.0.1:48653,tanh,-61.9547,12.3263,-6.11419,100,10.724,99,6.11419
objective_68c346b8,TERMINATED,127.0.0.1:48660,tanh,23.7916,2.108,2.83643,100,10.7375,99,-2.83643


Result for objective_56a79fce:
  date: 2022-07-22_15-48-27
  done: false
  experiment_id: 7046e554c1db465bbed80331b034b40f
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 20.0
  neg_mean_loss: -20.0
  node_ip: 127.0.0.1
  pid: 48590
  time_since_restore: 0.10438704490661621
  time_this_iter_s: 0.10438704490661621
  time_total_s: 0.10438704490661621
  timestamp: 1658501307
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 56a79fce
  warmup_time: 0.002905130386352539
  
Result for objective_583ee2ca:
  date: 2022-07-22_15-48-30
  done: false
  experiment_id: 59c76129c7074e41ab39b330d898ba03
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 8.0
  neg_mean_loss: -8.0
  node_ip: 127.0.0.1
  pid: 48605
  time_since_restore: 0.10483098030090332
  time_this_iter_s: 0.10483098030090332
  time_total_s: 0.10483098030090332
  timestamp: 1658501310
  timesteps_since_restore: 0
  training_iterati

Result for objective_58405952:
  date: 2022-07-22_15-48-41
  done: true
  experiment_id: 5902a2d532b3437e8b94d02919e1604c
  experiment_tag: 3_activation=relu,height=-87.8025,steps=100,width=19.6080
  hostname: Kais-MacBook-Pro.local
  iterations: 99
  iterations_since_restore: 100
  mean_loss: 1.271001380631752
  neg_mean_loss: -1.271001380631752
  node_ip: 127.0.0.1
  pid: 48606
  time_since_restore: 11.751991033554077
  time_this_iter_s: 0.1101231575012207
  time_total_s: 11.751991033554077
  timestamp: 1658501321
  timesteps_since_restore: 0
  training_iteration: 100
  trial_id: '58405952'
  warmup_time: 0.0027670860290527344
  
Result for objective_60d7fb56:
  date: 2022-07-22_15-48-44
  done: false
  experiment_id: 26d42c9dcc6e4354b3fa7355fc806522
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 14.954664969255782
  neg_mean_loss: -14.954664969255782
  node_ip: 127.0.0.1
  pid: 48634
  time_since_restore: 0.10394525527954102
  time_this

Result for objective_68c346b8:
  date: 2022-07-22_15-48-57
  done: false
  experiment_id: fe241f21de6a4466bcdf713da6d4cd92
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 12.37916148081603
  neg_mean_loss: -12.37916148081603
  node_ip: 127.0.0.1
  pid: 48660
  time_since_restore: 0.10358977317810059
  time_this_iter_s: 0.10358977317810059
  time_total_s: 0.10358977317810059
  timestamp: 1658501337
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 68c346b8
  warmup_time: 0.0028688907623291016
  
Result for objective_66bdb9e8:
  date: 2022-07-22_15-48-59
  done: false
  experiment_id: fae00fc97db044e9965698562b20039b
  hostname: Kais-MacBook-Pro.local
  iterations: 47
  iterations_since_restore: 48
  mean_loss: -6.025789125550826
  neg_mean_loss: 6.025789125550826
  node_ip: 127.0.0.1
  pid: 48653
  time_since_restore: 5.1440629959106445
  time_this_iter_s: 0.10767483711242676
  time_total_s: 5.1440629959106445
  timestamp: 165

And we again show the ideal hyperparameters.

In [16]:
print("Best hyperparameters found were: ", results.get_best_result().config)

Best hyperparameters found were:  {'steps': 100, 'width': 11.45060059898019, 'height': -66.52281754112352, 'activation': 'tanh'}


In [17]:
ray.shutdown()